In [1]:
# checkpoint_path = './train_log/train/model-15'
# checkpoint_path = '../train_log/train/checkpoint'
checkpoint_path = '/home/nam/tmp/01/train_log/train/checkpoint'

In [2]:
import VAE

OpenCV is built with OpenMP support. This usually results in poor performance. For details, see https://github.com/tensorpack/benchmarks/blob/master/ImageNet/benchmark-opencv-resize.py


In [3]:
from tensorpack.predict.config import PredictConfig
from tensorpack.tfutils import SmartInit

In [4]:
from VAE.Model import ModelDesc, Trainer  # , RandomZData
from VAE import get_default_hparams
# hps = get_default_hparams()
with open("./hps/hps.pickle", "rb") as input_file:
    hps = cPickle.load(input_file)
M = ModelDesc(hps)

In [5]:
pred_config = PredictConfig(
        session_init=SmartInit(checkpoint_path),
        model=M,
        input_names=['x'],
        output_names=['predict/y_pred']
)

In [6]:
from tensorpack.predict import OfflinePredictor
predictor = OfflinePredictor(pred_config)

[0923 18:35:13 @collection.py:146] New collections created in tower : tf.GraphKeys.MODEL_VARIABLES of size 2, tf.GraphKeys.METRIC_VARIABLES of size 2
[0923 18:35:13 @collection.py:165] These collections were modified but restored in : (tf.GraphKeys.SUMMARIES: 0->7)
[0923 18:35:13 @sessinit.py:87] WRN The following variables are in the checkpoint, but not found in the graph: global_step, optimize/beta1_power, optimize/beta2_power
[0923 18:35:14 @sessinit.py:114] Restoring checkpoint from /home/nam/tmp/01/train_log/train/model-330830 ...
INFO:tensorflow:Restoring parameters from /home/nam/tmp/01/train_log/train/model-330830


In [11]:
test_file = '/home/nam/data/ccxt/100_days_1m.csv'

import pandas as pd
import numpy as np
df = pd.read_csv(test_file)

In [12]:
cols = ['Open', 'High', 'Low', 'Close', 'Volume']
np_data = df[cols].values

In [9]:
# df_test = np.reshape(df_test, newshape = [hps.M, hps.T, hps.D])

In [13]:
i = np_data[-hps.T:]
input_np = np.asarray([i]*hps.M)

In [36]:
from VAE.load_data import test_segment
hps.scaler_path = '/home/nam/tmp/01/scaler/scaler.npz'
hps.normalize_data = 'min_max'
input_data = test_segment(df[cols], hps)

Inference Normalize: Min Max


In [37]:
i = input_data[-hps.M:]

In [38]:
# tf.TensorSpec((None, self.hps.T, self.hps.D), tf.float32, 'x'),
outputs = predictor(i)
outputs[0]

array([[[-1.        , -0.8687394 ]],

       [[-1.        , -0.86875665]],

       [[-1.        , -0.8687794 ]],

       ...,

       [[-1.        , -0.86881286]],

       [[-1.        , -0.8688456 ]],

       [[-1.        , -0.8688747 ]]], dtype=float32)

In [39]:
outputs[0].shape

(1024, 1, 2)

In [41]:
def softmax(x):
    "x shape is [N, Tau, C] when Tau == 1"
    assert np.shape(x)[1] == 1, "Tau is not 1"
    x = np.squeeze(x, axis=(1,))
    e_x = np.exp(x - np.max(x, axis = -1, keepdims = True))
    return e_x / e_x.sum(axis = -1, keepdims = True)

In [43]:
rs = softmax(outputs[0])
rs

array([[0.4672319 , 0.53276813],
       [0.46723616, 0.5327638 ],
       [0.46724185, 0.5327582 ],
       ...,
       [0.46725017, 0.53274983],
       [0.46725833, 0.53274167],
       [0.4672656 , 0.53273445]], dtype=float32)

In [46]:
np.any(rs[:,0] > rs[:,1])

False